In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pickle
import pandas as pd

from src.custom_inference_dataset import CustomInferenceDataset
from src.handler import Handler

In [4]:
with open('./data/x_num_categories_list.pkl', 'rb') as f:
	x_num_categories_list = pickle.load(f)

with open('./data/y_num_categories_list.pkl', 'rb') as f:
	y_num_categories_list = pickle.load(f)

with open('./data/label_encoders.pkl', 'rb') as f:
	label_encoders = pickle.load(f)

with open('./data/onehot_encoders.pkl', 'rb') as f:
	one_hot_encoders = pickle.load(f)

In [5]:
model = Handler(x_num_categories_list=x_num_categories_list, y_num_categories_list=y_num_categories_list)

model.load_state_dict(torch.load('./models/best_model.pth'))

model.eval()

C:\Users\Cai Selvas Sala\AppData\Local\Temp\ipykernel_25220\1632968379.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./models/best_mo

FileNotFoundError: [Errno 2] No such file or directory: './models/best_model.pth'

In [ ]:
image_folder_path = './data/archive/images/images'
x_test_file_path = './data/x_test.csv'

In [ ]:
test_dataset = CustomInferenceDataset(label_encoders=label_encoders, onehot_encoders=one_hot_encoders, x_test_path=x_test_file_path, image_folder_path=image_folder_path)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

resulting_df = pd.DataFrame(columns=['test_id', 'des_value'])

for image, tabular_data, attribute_name, test_id in test_loader:
	image = image.to(device)
	tabular_data = tabular_data.to(device)
	
	with torch.no_grad():
		try:
			pred = model(image, tabular_data)
		except Exception as e:
			print(f'Error predicting for test_id {test_id.item()}. Error message: {e}')
			resulting_df = resulting_df.append({'test_id': test_id.item(), 'des_value': 'INVALID'}, ignore_index=True)
			continue

	# Convert predictions to class indices for each attribute
	pred_classes = [torch.argmax(attr_pred, dim=1).item() for attr_pred in pred]
	
	pred_value = test_dataset.value_from_onehot_encoder(pred_classes[attribute_name], attribute_name)

    resulting_df = resulting_df.append({'test_id': test_id.item(), 'des_value': pred_value}, ignore_index=True)

# Guardar el DataFrame en un archivo CSV
resulting_df.to_csv('predictions.csv', index=False)